In [1]:
using Plots, ApproxFun, Interact
gr()  # Also works with plotly();

INFO: Recompiling stale cache file /Users/solver/.julia/lib/v0.6/ApproxFun.ji for module ApproxFun.


# $$u'' + 2u = \cos(\omega t)\qquad u(0)=u'(0)=0$$

Slide  to $\omega=\sqrt 2$ to see a resonance, where the solution grows

In [2]:
t = Fun(identity,0..1000) 
L = 𝒟^2+2I  # our differential operator, 𝒟 is equivalent to Derivative()

@manipulate for ω=0.:.1sqrt(2):2sqrt(2)
    u=[ivp();L]\[0.;0.;cos(ω*t)]
    plot(u)
end

Interact.Options{:SelectionSlider,Float64}(1: "input" = 1.2727922061357857 Float64 , "ω", 1.2727922061357857, "1.2727922061357857", Interact.OptionDict(DataStructures.OrderedDict("0.0"=>0.0,"0.14142135623730953"=>0.141421,"0.28284271247461906"=>0.282843,"0.42426406871192857"=>0.424264,"0.5656854249492381"=>0.565685,"0.7071067811865477"=>0.707107,"0.8485281374238571"=>0.848528,"0.9899494936611667"=>0.989949,"1.1313708498984762"=>1.13137,"1.2727922061357857"=>1.27279…), Dict(1.83848=>"1.838477631085024",0.282843=>"0.28284271247461906",2.68701=>"2.687005768508881",0.141421=>"0.14142135623730953",0.0=>"0.0",0.989949=>"0.9899494936611667",1.55563=>"1.5556349186104048",0.848528=>"0.8485281374238571",2.12132=>"2.121320343559643",1.27279=>"1.2727922061357857"…)), Any[], Any[], true, "horizontal")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 250 
 
 
 500 
 
 
 750 
 
 
 -4 
 
 
 -2 
 
 
 0 
 
 
 2 
 
 
 4 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 592.126,348.613 592.126,348.705 592.125,348.887 592.123,349.159 592.121,349.519 592.119,349.965 592.116,350.493 592.113,351.099 592.11,351.781 592.105,352.532 
 592.101,353.347 592.096,354.22 592.09,355.144 592.085,356.112 592.078,357.114 592.071,358.143 592.064,359.189 592.056,360.24 592.048,361.286 592.039,362.315 
 592.03,363.313 592.021,364.269 592.011,365.167 592,365.992 591.989,366.73 591.978,367.362 591.966,367.873 591.954,368.245 591.941,368.459 591.928,368.496 
 591.914,368.337 591.9,367.962 591.885,367.35 591.87,366.482 591.855,365.337 591.839,363.893 591.823,362.131 591.806,360.029 591.788,357.568 591.771,354.727 
 591.753,351.489 591.734,347.834 591.715,343.747 591.695,339.212 591.675,334.214 591.655,328.741 591.634,322.784 591.612,316.335 591.59,309.388 591.568,301.941 
 591.545,293.995 591.522,285.555 591.498,276.629 591.474,267.228 591.45,257.37 591.425,247.074 591.399,236.367 591.373,225.279 591.347,213.846 591.32,202.109 
 591.293,190.113 591.265,177.911 591.237,165.56 591.208,153.121 591.179,140.663 591.149,128.258 591.119,115.984 591.089,103.922 591.058,92.1581 591.027,80.7817 
 590.995,69.8847 590.962,59.5615 590.93,49.9076 590.896,41.0192 590.863,32.9917 590.829,25.9193 590.794,19.8932 590.759,15.0006 590.724,11.3238 590.688,8.93819 
 590.651,7.9114 590.614,8.30169 590.577,10.1566 590.539,13.5114 590.501,18.3881 590.463,24.7939 590.424,32.7201 590.384,42.1407 590.344,53.0123 590.304,65.2724 
 590.263,78.8396 590.221,93.6128 590.179,109.472 590.137,126.276 590.094,143.868 590.051,162.071 590.008,180.692 589.964,199.524 589.919,218.347 589.874,236.929 
 589.829,255.032 589.783,272.412 589.736,288.826 589.69,304.031 589.642,317.79 589.595,329.881 589.547,340.091 589.498,348.23 589.449,354.13 589.399,357.651 
 589.35,358.684 589.299,357.158 589.248,353.038 589.197,346.33 589.145,337.087 589.093,325.405 589.04,311.425 588.987,295.337 588.934,277.373 588.88,257.809 
 588.825,236.961 588.77,215.177 588.715,192.841 588.659,170.355 588.603,148.142 588.546,126.632 588.489,106.257 588.432,87.4369 588.373,70.5765 588.315,56.0507 
 588.256,44.1966 588.197,35.3041 588.137,29.607 588.077,27.2751 588.016,28.4074 587.955,33.0265 587.893,41.0749 587.831,52.4128 587.768,66.8183 587.705,83.9893 
 587.642,103.548 587.578,125.049 587.514,147.983 587.449,171.796 587.384,195.894 587.318,219.665 587.252,242.487 587.185,263.753 587.118,282.883 587.051,299.343 
 586.983,312.662 586.915,322.45 586.846,328.407 586.777,330.341 586.707,328.172 586.637,321.942 586.566,311.815 586.495,298.077 586.424,281.131 586.352,261.484 
 586.279,239.74 586.206,216.577 586.133,192.729 586.059,168.963 585.985,146.055 585.911,124.759 585.836,105.786 585.76,89.776 585.684,77.2706 585.608,68.6953 
 585.531,64.3397 585.454,64.3441 585.376,68.6912 585.298,77.2035 585.219,89.5473 585.14,105.242 585.061,123.678 584.981,144.135 584.9,165.813 584.82,187.861 
 584.738,209.409 584.657,229.608 584.574,247.664 584.492,262.87 584.409,274.638 584.325,282.528 584.241,286.266 584.157,285.758 584.072,281.095 583.987,272.552 
 583.901,260.576 583.815,245.764 583.728,228.837 583.641,210.61 583.554,191.945 583.466,173.717 583.378,156.768 583.289,141.867 583.2,129.667 583.11,120.68 
 583.02,115.244 582.929,113.512 582.838,115.443 582.747,120.806 582.655,129.196 582.563,140.055 582.47,152.708 582.377,166.404 582.283,180.356 582.189,193.788 
 582.095,205.984 582,216.326 581.904,224.331 581.808,229.676 581.712,232.215 581.615,231.985 581.518,229.195 581.421,224.209 581.323,217.518 581.224,209.698 
 581.125,201.368 581.026,193.145 580.926,185.596 580.826,179.199 580.725,174.307 580.624,171.125 580.523,169.7 580.421,169.919 580.318,171.528 580.216,174.158 
 580.112,177.363 580.009,180.663

# Bessel equation $$x^2 u'' + x u + (x^2 - \nu^2)u$$

In [4]:
x=Fun(identity,1..2000) 
d=domain(x)
B=Dirichlet()  
@manipulate for ν=0:.1:2000
	L=x^2*𝒟^2 + x*𝒟 + (x^2 - ν^2)   # our differential operator
   	u=[B;L]\[[besselj(ν,first(d)),besselj(ν,last(d))],0]
    plot(u;ylims=(-.2,.2))
end

Interact.Options{:SelectionSlider,Float64}(5: "input-2" = 1000.0 Float64 , "ν", 1000.0, "1000.0", Interact.OptionDict(DataStructures.OrderedDict("0.0"=>0.0,"0.1"=>0.1,"0.2"=>0.2,"0.3"=>0.3,"0.4"=>0.4,"0.5"=>0.5,"0.6"=>0.6,"0.7"=>0.7,"0.8"=>0.8,"0.9"=>0.9…), Dict(1929.8=>"1929.8",593.6=>"593.6",1580.5=>"1580.5",675.4=>"675.4",1333.0=>"1333.0",71.2=>"71.2",1032.4=>"1032.4",84.5=>"84.5",1754.2=>"1754.2",1579.3=>"1579.3"…)), Any[], Any[], true, "horizontal")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 500 
 
 
 1000 
 
 
 1500 
 
 
 -0.2 
 
 
 -0.1 
 
 
 0.0 
 
 
 0.1 
 
 
 0.2 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 592.126,176.14 592.126,176.143 592.126,176.15 592.126,176.159 592.125,176.171 592.125,176.187 592.124,176.205 592.124,176.227 592.123,176.252 592.122,176.28 
 592.122,176.312 592.121,176.346 592.12,176.384 592.119,176.425 592.118,176.47 592.116,176.518 592.115,176.569 592.114,176.623 592.112,176.681 592.111,176.743 
 592.109,176.808 592.108,176.876 592.106,176.948 592.104,177.024 592.102,177.103 592.1,177.186 592.098,177.273 592.096,177.364 592.094,177.459 592.091,177.557 
 592.089,177.66 592.086,177.767 592.084,177.877 592.081,177.992 592.078,178.111 592.076,178.235 592.073,178.362 592.07,178.494 592.067,178.631 592.064,178.772 
 592.06,178.917 592.057,179.067 592.054,179.222 592.05,179.382 592.047,179.546 592.043,179.715 592.04,179.889 592.036,180.068 592.032,180.251 592.028,180.44 
 592.024,180.634 592.02,180.833 592.016,181.037 592.012,181.246 592.007,181.461 592.003,181.681 591.998,181.906 591.994,182.136 591.989,182.371 591.985,182.612 
 591.98,182.858 591.975,183.11 591.97,183.367 591.965,183.629 591.96,183.897 591.955,184.169 591.949,184.448 591.944,184.731 591.939,185.02 591.933,185.313 
 591.927,185.612 591.922,185.916 591.916,186.225 591.91,186.539 591.904,186.858 591.898,187.182 591.892,187.51 591.886,187.843 591.88,188.18 591.874,188.522 
 591.867,188.867 591.861,189.217 591.854,189.571 591.848,189.928 591.841,190.289 591.834,190.654 591.827,191.021 591.82,191.392 591.813,191.765 591.806,192.141 
 591.799,192.519 591.792,192.898 591.784,193.28 591.777,193.663 591.769,194.047 591.762,194.432 591.754,194.817 591.746,195.203 591.738,195.588 591.731,195.973 
 591.723,196.356 591.715,196.739 591.706,197.119 591.698,197.498 591.69,197.874 591.681,198.247 591.673,198.616 591.664,198.982 591.656,199.343 591.647,199.699 
 591.638,200.049 591.629,200.394 591.621,200.733 591.612,201.064 591.602,201.388 591.593,201.704 591.584,202.012 591.575,202.31 591.565,202.598 591.556,202.877 
 591.546,203.144 591.536,203.4 591.527,203.645 591.517,203.876 591.507,204.095 591.497,204.3 591.487,204.49 591.477,204.666 591.467,204.826 591.456,204.97 
 591.446,205.098 591.435,205.209 591.425,205.302 591.414,205.377 591.404,205.433 591.393,205.47 591.382,205.488 591.371,205.485 591.36,205.462 591.349,205.418 
 591.338,205.353 591.327,205.266 591.315,205.157 591.304,205.026 591.292,204.872 591.281,204.695 591.269,204.495 591.257,204.272 591.246,204.025 591.234,203.755 
 591.222,203.461 591.21,203.144 591.197,202.803 591.185,202.438 591.173,202.05 591.161,201.639 591.148,201.205 591.136,200.748 591.123,200.268 591.11,199.767 
 591.098,199.244 591.085,198.7 591.072,198.135 591.059,197.551 591.046,196.947 591.033,196.325 591.019,195.685 591.006,195.028 590.993,194.355 590.979,193.668 
 590.965,192.966 590.952,192.252 590.938,191.525 590.924,190.789 590.91,190.043 590.896,189.29 590.882,188.53 590.868,187.765 590.854,186.997 590.84,186.226 
 590.825,185.455 590.811,184.686 590.797,183.92 590.782,183.158 590.767,182.402 590.752,181.655 590.738,180.918 590.723,180.193 590.708,179.481 590.693,178.785 
 590.678,178.106 590.662,177.447 590.647,176.808 590.632,176.193 590.616,175.602 590.601,175.039 590.585,174.503 590.569,173.998 590.553,173.525 590.538,173.086 
 590.522,172.681 590.506,172.314 590.489,171.985 590.473,171.696 590.457,171.448 590.441,171.243 590.424,171.081 590.408,170.964 590.391,170.893 590.374,170.868 
 590.358,170.89 590.341,170.96 590.324,171.079 590.307,171.247 590.29,171.463 590.273,171.728 590.256,172.042 590.238,172.405 590.221,172.816 590.203,173.275 
 590.186,173.78 590.168,174.332 590.151,174.928 590.133,175.568 590.115,176.25 590.097,176.973 590.079,177.734 590.061,178.532 590.043,179.365 590.025,180.23 
 590.006,181.125 5

# Exponentially ill-conditioned Lee & Greengard BVP $\epsilon u'' - x u' + u = 0$

In [5]:
x=Fun(identity)
B=Dirichlet()

@manipulate for ϵ=0.0005:0.0005:0.4
    L=ϵ*𝒟^2 - x*𝒟 + I

     u=[B;L]\[[1.,2.],0.]
    plot(u;ylims=(-2.,3.))
end

Interact.Options{:SelectionSlider,Float64}(9: "input-3" = 0.2 Float64 , "ϵ", 0.2, "0.2", Interact.OptionDict(DataStructures.OrderedDict("0.0005"=>0.0005,"0.001"=>0.001,"0.0015"=>0.0015,"0.002"=>0.002,"0.0025"=>0.0025,"0.003"=>0.003,"0.0035"=>0.0035,"0.004"=>0.004,"0.0045"=>0.0045,"0.005"=>0.005…), Dict(0.259=>"0.259",0.117=>"0.117",0.0305=>"0.0305",0.0225=>"0.0225",0.2935=>"0.2935",0.0655=>"0.0655",0.221=>"0.221",0.1375=>"0.1375",0.0635=>"0.0635",0.2375=>"0.2375"…)), Any[], Any[], true, "horizontal")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.5 
 
 
 0.0 
 
 
 0.5 
 
 
 -2 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 592.126,80.0271 592.012,80.2441 591.785,80.6769 591.445,81.3235 590.991,82.1808 590.425,83.2446 589.745,84.5099 588.953,85.9707 588.049,87.6202 587.032,89.451 
 585.905,91.4549 584.666,93.623 583.317,95.946 581.858,98.4143 580.29,101.018 578.613,103.746 576.828,106.589 574.935,109.536 572.937,112.576 570.832,115.7 
 568.623,118.896 566.31,122.155 563.894,125.467 561.376,128.823 558.757,132.213 556.039,135.629 553.222,139.062 550.307,142.505 547.295,145.95 544.189,149.391 
 540.989,152.821 537.696,156.234 534.312,159.624 530.839,162.987 527.277,166.318 523.628,169.613 519.893,172.867 516.075,176.079 512.175,179.243 508.193,182.359 
 504.133,185.424 499.996,188.435 495.782,191.391 491.495,194.289 487.136,197.13 482.706,199.912 478.207,202.633 473.642,205.293 469.012,207.892 464.319,210.429 
 459.565,212.903 454.752,215.314 449.882,217.662 444.956,219.947 439.978,222.169 434.949,224.327 429.87,226.422 424.745,228.454 419.575,230.422 414.362,232.327 
 409.109,234.169 403.817,235.947 398.49,237.663 393.128,239.315 387.734,240.904 382.311,242.43 376.861,243.893 371.385,245.293 365.887,246.63 360.368,247.904 
 354.831,249.114 349.277,250.261 343.71,251.345 338.132,252.365 332.544,253.321 326.949,254.213 321.35,255.041 315.748,255.804 310.146,256.503 304.547,257.136 
 298.952,257.704 293.365,258.206 287.786,258.642 282.219,259.011 276.665,259.313 271.128,259.548 265.609,259.714 260.111,259.812 254.635,259.841 249.185,259.8 
 243.762,259.689 238.368,259.508 233.006,259.255 227.679,258.93 222.387,258.532 217.134,258.061 211.921,257.516 206.751,256.897 201.626,256.203 196.548,255.433 
 191.518,254.588 186.54,253.665 181.614,252.665 176.744,251.588 171.931,250.433 167.177,249.199 162.484,247.887 157.854,246.497 153.289,245.028 148.79,243.48 
 144.361,241.855 140.001,240.152 135.714,238.371 131.5,236.515 127.363,234.584 123.303,232.579 119.321,230.502 115.421,228.355 111.603,226.14 107.868,223.86 
 104.22,221.517 100.658,219.116 97.1838,216.66 93.7999,214.152 90.5072,211.599 87.307,209.004 84.2007,206.374 81.1894,203.714 78.2745,201.032 75.4572,198.334 
 72.7386,195.627 70.1198,192.921 67.6019,190.222 65.186,187.541 62.8729,184.885 60.6638,182.265 58.5594,179.691 56.5607,177.172 54.6685,174.719 52.8835,172.342 
 51.2065,170.051 49.6381,167.857 48.1791,165.77 46.83,163.799 45.5914,161.954 44.4637,160.244 43.4475,158.679 42.5432,157.265 41.751,156.011 41.0715,154.923 
 40.5047,154.007 40.051,153.268 39.7106,152.71 39.4836,152.337 39.3701,152.149 
 "/>
 
 
 
 
 y1

# Piecewise $$u'' + \omega u \begin{cases} 1 & |x| < c\cr 0 & {\rm otherwise}\end{cases}  = 0,\qquad u(-1)=1,u(1)=0$$

In [32]:
@manipulate for ω=1:50000,c=0.1:0.01:1.
    f=Fun(x->abs(x)<c?1:0,Domain(-1..1) \ [-c,c])
    S=space(f)
    B=[Dirichlet(S);continuity(S,0:1)]
    plot([B;𝒟^2+ω*f]\[[1.,0.],zeros(4),0];ylims=(-0.5,1.1))
end

Interact.Options{:SelectionSlider,Int64}(33: "input-14" = 25000 Int64 , "ω", 25000, "25000", Interact.OptionDict(DataStructures.OrderedDict("1"=>1,"2"=>2,"3"=>3,"4"=>4,"5"=>5,"6"=>6,"7"=>7,"8"=>8,"9"=>9,"10"=>10…), Dict(43031=>"43031",22035=>"22035",39001=>"39001",48586=>"48586",39722=>"39722",43089=>"43089",19698=>"19698",29981=>"29981",43977=>"43977",7237=>"7237"…)), Any[], Any[], true, "horizontal")

Interact.Options{:SelectionSlider,Float64}(35: "input-15" = 0.55 Float64 , "c", 0.55, "0.55", Interact.OptionDict(DataStructures.OrderedDict("0.1"=>0.1,"0.11"=>0.11,"0.12"=>0.12,"0.13"=>0.13,"0.14"=>0.14,"0.15"=>0.15,"0.16"=>0.16,"0.17"=>0.17,"0.18"=>0.18,"0.19"=>0.19…), Dict(0.54=>"0.54",0.86=>"0.86",0.35=>"0.35",0.22=>"0.22",0.38=>"0.38",0.56=>"0.56",0.2=>"0.2",0.89=>"0.89",0.48=>"0.48",0.49=>"0.49"…)), Any[], Any[], true, "horizontal")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.5 
 
 
 0.0 
 
 
 0.5 
 
 
 -0.5 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 163.74,257.336 163.737,257.331 163.731,257.321 163.723,257.306 163.712,257.286 163.698,257.261 163.682,257.231 163.663,257.196 163.641,257.156 163.616,257.111 
 163.589,257.061 163.559,257.006 163.526,256.946 163.49,256.881 163.452,256.811 163.411,256.736 163.367,256.656 163.32,256.571 163.271,256.481 163.219,256.386 
 163.164,256.286 163.107,256.182 163.047,256.072 162.984,255.957 162.918,255.838 162.85,255.713 162.779,255.584 162.706,255.449 162.629,255.31 162.55,255.166 
 162.468,255.016 162.384,254.862 162.297,254.703 162.207,254.54 162.115,254.371 162.019,254.197 161.922,254.019 161.821,253.835 161.718,253.647 161.612,253.454 
 161.504,253.256 161.393,253.054 161.279,252.846 161.163,252.634 161.044,252.417 160.922,252.195 160.798,251.968 160.671,251.737 160.541,251.501 160.409,251.26 
 160.275,251.014 160.137,250.763 159.998,250.508 159.855,250.249 159.71,249.984 159.563,249.715 159.413,249.441 159.26,249.162 159.105,248.879 158.947,248.591 
 158.787,248.299 158.624,248.002 158.459,247.7 158.291,247.394 158.12,247.083 157.948,246.768 157.772,246.448 157.595,246.124 157.414,245.795 157.232,245.461 
 157.046,245.124 156.859,244.781 156.669,244.434 156.476,244.083 156.281,243.727 156.084,243.367 155.884,243.003 155.682,242.634 155.477,242.261 155.271,241.883 
 155.061,241.502 154.85,241.115 154.636,240.725 154.419,240.33 154.201,239.931 153.98,239.528 153.756,239.121 153.531,238.709 153.303,238.293 153.073,237.873 
 152.84,237.449 152.606,237.021 152.369,236.589 152.129,236.152 151.888,235.712 151.644,235.267 151.398,234.818 151.15,234.366 150.9,233.909 150.648,233.449 
 150.393,232.984 150.136,232.516 149.877,232.043 149.616,231.567 149.353,231.087 149.088,230.603 148.821,230.115 148.551,229.624 148.28,229.128 148.006,228.629 
 147.731,228.126 147.453,227.62 147.173,227.109 146.892,226.595 146.608,226.078 146.322,225.556 146.034,225.032 145.745,224.503 145.453,223.971 145.16,223.436 
 144.864,222.897 144.567,222.354 144.268,221.808 143.967,221.259 143.664,220.706 143.359,220.15 143.052,219.59 142.744,219.027 142.433,218.461 142.121,217.891 
 141.807,217.319 141.492,216.743 141.174,216.163 140.855,215.581 140.534,214.995 140.211,214.407 139.887,213.815 139.561,213.22 139.233,212.622 138.904,212.021 
 138.573,211.417 138.24,210.81 137.906,210.2 137.57,209.587 137.232,208.971 136.893,208.352 136.552,207.731 136.21,207.107 135.867,206.479 135.521,205.85 
 135.175,205.217 134.826,204.582 134.477,203.944 134.126,203.303 133.773,202.66 133.419,202.014 133.064,201.365 132.707,200.715 132.349,200.061 131.989,199.405 
 131.629,198.747 131.266,198.086 130.903,197.423 130.538,196.757 130.172,196.089 129.805,195.419 129.436,194.747 129.066,194.072 128.695,193.395 128.323,192.716 
 127.95,192.035 127.575,191.351 127.199,190.666 126.823,189.978 126.445,189.288 126.066,188.597 125.686,187.903 125.304,187.208 124.922,186.51 124.539,185.811 
 124.155,185.11 123.769,184.407 123.383,183.702 122.996,182.996 122.608,182.287 122.219,181.578 121.829,180.866 121.438,180.153 121.046,179.438 120.653,178.721 
 120.26,178.004 119.865,177.284 119.47,176.563 119.074,175.841 118.678,175.117 118.28,174.392 117.882,173.665 117.483,172.938 117.084,172.209 116.683,171.478 
 116.282,170.747 115.881,170.014 115.479,169.28 115.076,168.545 114.672,167.809 114.268,167.072 113.864,166.334 113.459,165.594 113.053,164.854 112.647,164.113 
 112.24,163.371 111.833,162.628 111.426,161.885 111.017,161.14 110.609,160.395 110.2,159.649 109.791,158.903 109.381,158.155 108.972,157.407 108.561,156.659 
 108.151,155.91 107.74,155.16 107.329,154.41 106.917,153.66 106.506,152.909 106.094,152.157 105.682,151.405 105.27,150.653 104.857,149.901 104.445,149.148 
 104.032,148.395 103.62,147.642 10

# Forced Helmholtz equation $$u_{xx} + u_{yy} + k u = {\rm e}^{-(x-x_0)^2-(y-y_0)^2},\qquad \partial u = 0$$

In [33]:
plotly()

d=Domain(-1..1)^2
B=Dirichlet(d)
Δ=Laplacian(d)

k = slider(-100.:.1:100.; label="k")
x₀ = slider(-2.:0.1:2.; label="x₀")
y₀ = slider(-2.:0.1:2.; label="y₀")

QR = map(k->qrfact([B;Δ+k*I]),signal(k))
f = map((x0,y0)->Fun((x,y)->exp(-(x-x0)^2-(y-y0)^2)), signal(x₀), signal(y₀)) 
u = map((QR,f)->\(QR,[0.0;f];tolerance=1E-4),QR,f)

map(display, [k,x₀,y₀]) #display widgets
display(map(contour,u)) #display results;

LoadError: [91mUndefVarError: banded_dense_axpy! not defined[39m

In [8]:
plotly()

d=Domain(-1..1)^2
B=[Dirichlet(d[1])⊗I;I⊗ldirichlet(d[2]);I⊗rneumann(d[2])]
Δ=Laplacian(d)

k = slider(-100.:.1:100.; label="k")
x₀ = slider(-2.:0.1:2.; label="x₀")
y₀ = slider(-2.:0.1:2.; label="y₀")

QR = map(k->qrfact([B;Δ+k*I]),signal(k))
f = map((x0,y0)->Fun((x,y)->exp(-(x-x0)^2-(y-y0)^2),d), signal(x₀), signal(y₀)) 
u = map((QR,f)->\(QR,[0.0;0.0;0.0;0.0;f];tolerance=1E-4),QR,f)

map(display, [k,x₀,y₀]) #display widgets
display(map(contour,u)) #display results;

Interact.Slider{Float64}(36: "input-11" = 0.0 Float64 ,"k",0.0,-100.0:0.1:100.0,"horizontal",true,".3f",true)

Interact.Slider{Float64}(38: "input-12" = 0.0 Float64 ,"x₀",0.0,-2.0:0.1:2.0,"horizontal",true,".3f",true)

Interact.Slider{Float64}(40: "input-13" = 0.0 Float64 ,"y₀",0.0,-2.0:0.1:2.0,"horizontal",true,".3f",true)

#     Dirichlet Helmholtz equation $$\Delta u + k u = 0, u(\pm 1,y)=u(x,\pm 1) = 1$$

In [9]:
d=Domain(-1..1)^2
B=Dirichlet(d)
Δ=Laplacian(d)

@manipulate for k=-200.0:0.001:200.0
    contour(\([B;Δ+k*I],[ones(∂(d));0.];tolerance=1E-3))
end

Interact.Options{:SelectionSlider,Float64}(47: "input-14" = 0.0 Float64 ,"k",0.0,"0.0",Interact.OptionDict(DataStructures.OrderedDict("-200.0"=>-200.0,"-199.999"=>-199.999,"-199.998"=>-199.998,"-199.997"=>-199.997,"-199.996"=>-199.996,"-199.995"=>-199.995,"-199.994"=>-199.994,"-199.993"=>-199.993,"-199.992"=>-199.992,"-199.991"=>-199.991…),Dict(-173.635=>"-173.635",-55.429=>"-55.429",81.147=>"81.147",-169.56=>"-169.56",-102.913=>"-102.913",-110.907=>"-110.907",42.161=>"42.161",124.513=>"124.513",-28.394=>"-28.394",-176.318=>"-176.318"…)),Any[],Any[],true,"horizontal")

# Convection diffusion $u_t = \epsilon u_{xx} + (B+C x) u_x, u(x,0)=e^{-20x^2}, u(-1,t)=u(1,t) = 0$

In [15]:
dx=Domain(-1..1);dt=Domain(0..1)
d=dx*dt
Dx=Derivative(d,[1,0]);Dt=Derivative(d,[0,1])
x,t=Fun(dx*dt)
Bcs=timedirichlet(d)
u0=chop!(Fun(x->exp(-20x^2),dx),1E-12)
@manipulate for ε=0.001:0.001:2.,B=-5.:0.1:5.,C=-5.:0.1:5.
    V=B+C*x    
    u=\([Bcs;Dt-ε*Dx^2-V*Dx],[u0;zeros(3)];tolerance=1E-3,maxlength=2000)
    contour(u;xlims=(-1.,1.),ylims=(0.,1.))
end

Interact.Options{:SelectionSlider,Float64}(59: "input-18" = 1.0 Float64 ,"ε",1.0,"1.0",Interact.OptionDict(DataStructures.OrderedDict("0.001"=>0.001,"0.002"=>0.002,"0.003"=>0.003,"0.004"=>0.004,"0.005"=>0.005,"0.006"=>0.006,"0.007"=>0.007,"0.008"=>0.008,"0.009"=>0.009,"0.01"=>0.01…),Dict(0.259=>"0.259",1.268=>"1.268",0.117=>"0.117",1.861=>"1.861",1.059=>"1.059",0.849=>"0.849",0.875=>"0.875",1.19=>"1.19",1.174=>"1.174",1.113=>"1.113"…)),Any[],Any[],true,"horizontal")

Interact.Options{:SelectionSlider,Float64}(61: "input-19" = 0.0 Float64 ,"B",0.0,"0.0",Interact.OptionDict(DataStructures.OrderedDict("-5.0"=>-5.0,"-4.9"=>-4.9,"-4.8"=>-4.8,"-4.7"=>-4.7,"-4.6"=>-4.6,"-4.5"=>-4.5,"-4.4"=>-4.4,"-4.3"=>-4.3,"-4.2"=>-4.2,"-4.1"=>-4.1…),Dict(1.2=>"1.2",4.3=>"4.3",-2.3=>"-2.3",2.0=>"2.0",1.5=>"1.5",-1.3=>"-1.3",-3.4=>"-3.4",1.4=>"1.4",-2.2=>"-2.2",-3.0=>"-3.0"…)),Any[],Any[],true,"horizontal")

Interact.Options{:SelectionSlider,Float64}(63: "input-20" = 0.0 Float64 ,"C",0.0,"0.0",Interact.OptionDict(DataStructures.OrderedDict("-5.0"=>-5.0,"-4.9"=>-4.9,"-4.8"=>-4.8,"-4.7"=>-4.7,"-4.6"=>-4.6,"-4.5"=>-4.5,"-4.4"=>-4.4,"-4.3"=>-4.3,"-4.2"=>-4.2,"-4.1"=>-4.1…),Dict(1.2=>"1.2",4.3=>"4.3",-2.3=>"-2.3",2.0=>"2.0",1.5=>"1.5",-1.3=>"-1.3",-3.4=>"-3.4",1.4=>"1.4",-2.2=>"-2.2",-3.0=>"-3.0"…)),Any[],Any[],true,"horizontal")

# Schrodinger equation $$i \epsilon u_t = -{\epsilon^2 \over 2} u_{xx} + x^2 u$$

In [16]:
dx=Domain(0..1);dt=Domain(0..0.54)
d=dx*dt
x,t=Fun(d)
V=x^2

Dt=Derivative(d,[0,1]);Dx=Derivative(d,[1,0])

@manipulate for ϵ=0.005:0.002:0.3
    u0=Fun(x->exp(-25*(x-0.5)^2)*exp(-1.0im/(5*ϵ)*log(2cosh(5*(x-0.5)))),dx)
    L=1im*ϵ*Dt+0.5*ϵ^2*Dx^2-V

    u=\([timedirichlet(d);L],[u0;zeros(3)];tolerance=1E-3,maxlength=2000)
    contour(real(u))
end

Interact.Options{:SelectionSlider,Float64}(67: "input-21" = 0.151 Float64 ,"ϵ",0.151,"0.151",Interact.OptionDict(DataStructures.OrderedDict("0.005"=>0.005,"0.007"=>0.007,"0.009000000000000001"=>0.009,"0.011"=>0.011,"0.013000000000000001"=>0.013,"0.015"=>0.015,"0.017"=>0.017,"0.019"=>0.019,"0.021"=>0.021,"0.023000000000000003"=>0.023…),Dict(0.053=>"0.053",0.209=>"0.20900000000000002",0.217=>"0.217",0.055=>"0.055",0.129=>"0.129",0.259=>"0.259",0.117=>"0.117",0.211=>"0.21100000000000002",0.203=>"0.203",0.225=>"0.225"…)),Any[],Any[],true,"horizontal")

# Nonlinear BVP $$\epsilon u'' + 6(1-x^2)u' +u^2=1$$ $$u(-1)=u(1)=0$$

In [18]:
x=Fun()
u0=0.0x
@manipulate for ε=0.001:0.001:1.,c=0.:0.1:1.
    N=u->[u(-1.)-c;u(1.);ε*u''+6*(1-x^2)*u'+u^2-1.]

    u=newton(N,u0)
    plot(u;ylims=(-1.,1.))
end

Interact.Options{:SelectionSlider,Float64}(77: "input-24" = 0.5 Float64 ,"ε",0.5,"0.5",Interact.OptionDict(DataStructures.OrderedDict("0.001"=>0.001,"0.002"=>0.002,"0.003"=>0.003,"0.004"=>0.004,"0.005"=>0.005,"0.006"=>0.006,"0.007"=>0.007,"0.008"=>0.008,"0.009"=>0.009,"0.01"=>0.01…),Dict(0.259=>"0.259",0.117=>"0.117",0.849=>"0.849",0.875=>"0.875",0.785=>"0.785",0.89=>"0.89",0.221=>"0.221",0.756=>"0.756",0.658=>"0.658",0.449=>"0.449"…)),Any[],Any[],true,"horizontal")

Interact.Options{:SelectionSlider,Float64}(79: "input-25" = 0.5 Float64 ,"c",0.5,"0.5",Interact.OptionDict(DataStructures.OrderedDict("0.0"=>0.0,"0.1"=>0.1,"0.2"=>0.2,"0.3"=>0.3,"0.4"=>0.4,"0.5"=>0.5,"0.6"=>0.6,"0.7"=>0.7,"0.8"=>0.8,"0.9"=>0.9…),Dict(0.6=>"0.6",0.3=>"0.3",0.7=>"0.7",0.0=>"0.0",0.2=>"0.2",0.9=>"0.9",0.8=>"0.8",0.5=>"0.5",0.1=>"0.1",0.4=>"0.4"…)),Any[],Any[],true,"horizontal")